<h1>Procedure Contracts with no reference for provider in provider table</h1>

<ul>
    <li>123'881 total records in table regardless of any problem</li>
    <li>1'634 records (~1.3%) can not find a reference in the provider table. Those account for 1'122 (~3.5%) distinct providers</li>
</ul>

In [1]:
SELECT COUNT(*) FROM [cleandata].[sicop_procedure_contract] AS pc

(1 row affected)

Total execution time: 00:00:00.124

(No column name)
123881


In [4]:
SELECT 
    COUNT(*)  AS Total_Records_Orphan_Cedula
    ,COUNT(DISTINCT CONCAT(pc.Número_procedimiento, '|', pc.Número_contrato))  AS Distinct_Orphan_Nk
    ,COUNT(DISTINCT pc.Cédula_adjudicatario)  AS Distinct_Orphan_Cedula
    ,COUNT(DISTINCT pc.Número_procedimiento)  AS Distinct_Orphan_ProcedureNum
FROM [cleandata].[sicop_procedure_contract] AS pc
LEFT OUTER JOIN cleandata.sicop_provider AS p ON pc.Cédula_adjudicatario = p.Cedula
WHERE p.Cedula IS NULL

(1 row affected)

Total execution time: 00:00:00.272

Total_Records_Orphan_Cedula,Distinct_Orphan_Nk,Distinct_Orphan_Cedula,Distinct_Orphan_ProcedureNum
1634,1634,1122,1434


<h1>Categories with references in the hierarchy relation to other categories</h1>

<ul>
    <li>48'585 categories of level 4 in table regardless of any problem </li>
    <li>4'881 categories of level 3 in table regardless of any problem</li>
    <li>469 categories of level 2 in table regardless of any problem</li>
    <li>57 categories of level 1 in table regardless of any problem</li>
    <li>0 orphan references in any of the categories hierarchies </li>
</ul>

In [5]:
SELECT count(*) FROM cleandata.sicop_categorylevel4
SELECT count(*) FROM cleandata.sicop_categorylevel3
SELECT count(*) FROM cleandata.sicop_categorylevel2
SELECT count(*) FROM cleandata.sicop_categorylevel1

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.021

(No column name)
48585


(No column name)
4881


(No column name)
469


(No column name)
57


In [7]:
SELECT 
    count(*) AS Null_Records
FROM cleandata.sicop_categorylevel4 AS cat4
LEFT OUTER JOIN cleandata.sicop_categorylevel3 AS cat3 ON LEFT(cat4.category_id, 6) = cat3.category_id
WHERE cat3.category_id IS NULL

SELECT 
    count(*) AS Null_Records
FROM cleandata.sicop_categorylevel3 AS cat3
LEFT OUTER JOIN cleandata.sicop_categorylevel2 AS cat2 ON LEFT(cat3.category_id, 4) = cat2.category_id
WHERE cat2.category_id IS NULL

SELECT 
    count(*) AS Null_Records
FROM cleandata.sicop_categorylevel2 AS cat2
LEFT OUTER JOIN cleandata.sicop_categorylevel1 AS cat1 ON LEFT(cat2.category_id, 2) = cat1.category_id
WHERE cat1.category_id IS NULL

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.043

Null_Records
0


Null_Records
0


Null_Records
0


<h1>Procedure participations map table with orphan providers or procedures</h1>

<ul>
    <li>361'973 total participation records (provider + procedure_num) in table regardless of any problem</li>
    <li>100'581 total procedure_num regardless of any problem</li>
    <li>35'341 total participation records (~10%) do not have a procedure num in procedure contracts table. This is for 11'260 distinct procedures (~12%)</li>
</ul>

In [11]:
SELECT count(*) FROM cleandata.sicop_procedure_participations_per_provider

SELECT count(distinct procedure_num) FROM cleandata.sicop_procedure_participations_per_provider

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.517

(No column name)
361973


(No column name)
100581


In [9]:
SELECT 
    count(*) AS Null_Records
FROM cleandata.sicop_procedure_participations_per_provider AS par
LEFT OUTER JOIN cleandata.sicop_provider AS p ON par.provider_id = p.Cedula
WHERE p.Cedula IS NULL

(1 row affected)

Total execution time: 00:00:00.499

Null_Records
0


In [13]:
SELECT 
    count(*) AS Null_Records
    ,count(DISTINCT par.procedure_num) AS Null_Procedure_Num
FROM cleandata.sicop_procedure_participations_per_provider AS par
LEFT OUTER JOIN (SELECT DISTINCT Número_procedimiento AS procedure_num FROM cleandata.sicop_procedure_contract) AS pc ON par.procedure_num = pc.procedure_num
WHERE pc.procedure_num IS NULL

(1 row affected)

Total execution time: 00:00:01.447

Null_Records,Null_Procedure_Num
35341,11260


<h1>Procedure contracts with no records in participations map table</h1>

<ul>
    <li>123'881 distinct combinations of procedure_num+contact_num in procedure contracts</li>
    <li>89'950 distinct procedure nums, and 10'813 distinct provider_id in procedure_contract table regardless of any problem</li>
    <li>629 distinct procedure nums (~0.7%) in procedure_contract table can not be found in participation map table</li>
    <li>1'506 (~14%) distinct provider_id in procedure_contract table can not be found in participation map table</li>
</ul>

In [25]:
SELECT 
    count(*) AS Records
FROM cleandata.sicop_procedure_contract AS pc

SELECT 
    count(*) AS Records_procedure_num
FROM (SELECT DISTINCT Número_procedimiento AS procedure_num FROM cleandata.sicop_procedure_contract) AS pc

SELECT 
    count(*) AS Records_Cedula
FROM (SELECT DISTINCT Cédula_adjudicatario AS procedure_num FROM cleandata.sicop_procedure_contract) AS pc

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.369

Records
123881


Records_procedure_num
89950


Records_Cedula
10813


In [20]:
SELECT 
    count(*) AS Null_Records
    ,count(DISTINCT pc.Número_procedimiento) AS Null_Procedure_Num
FROM cleandata.sicop_procedure_contract AS pc
LEFT OUTER JOIN (SELECT DISTINCT procedure_num FROM cleandata.sicop_procedure_participations_per_provider) AS par ON pc.Número_procedimiento = par.procedure_num
WHERE par.procedure_num IS NULL

(1 row affected)

Total execution time: 00:00:01.277

Null_Records,Null_Procedure_Num
867,629


In [23]:
SELECT 
    count(*) AS Null_Records
    ,count(DISTINCT pc.Número_procedimiento) AS Null_Procedure_Num
    ,count(DISTINCT pc.Cédula_adjudicatario) AS Null_Cedula_Num
FROM cleandata.sicop_procedure_contract AS pc
LEFT OUTER JOIN (SELECT DISTINCT procedure_num, provider_id FROM cleandata.sicop_procedure_participations_per_provider) AS par ON pc.Número_procedimiento = par.procedure_num AND pc.Cédula_adjudicatario = par.provider_id
WHERE par.procedure_num IS NULL

(1 row affected)

Total execution time: 00:00:02.417

Null_Records,Null_Procedure_Num,Null_Cedula_Num
2191,1760,1506
